# Hidden Markov Models (HMMs)

HMMs can be effective in solving problems in reinforcement learning and temporal pattern recognition. We will implement such models to predict the weekly price of corn from the dataset [_Weekly Corn Prices_](https://www.kaggle.com/nickwong64/corn2015-2017).  

The file composed of simply 2 columns. One is the date (weekend) and the other is corn close price. The period is from 2015-01-04 to 2017-10-01. The original data is downloaded from Quantopian corn futures price.

In [1]:
import numpy as np
import pandas as pd
from collections import namedtuple
from itertools import permutations, chain, product
from sklearn.cluster import KMeans
from inspect import Signature, Parameter

# Data 

In [2]:
CORN_2013_2017 = 'corn2013-2017.txt'
CORN_2015_2017 = 'corn2015-2017.txt'
OHL = 'corn_OHLC2013-2017.txt'

In [3]:
corn13_17 = pd.read_csv(CORN_2013_2017, names = ("week","price") )
corn15_17 = pd.read_csv(CORN_2015_2017, names = ("week","price"))
OHL_df = pd.read_csv(OHL, names = ("week","open","high","low","close"))

We will start using the data set `corn13_17`

In [4]:
corn13_17

week     price
0    2013-01-06  7.794975
1    2013-01-13  7.863400
2    2013-01-20  8.234920
3    2013-01-27  8.186260
4    2013-02-03  8.317480
..          ...       ...
243  2017-09-03  3.512500
244  2017-09-10  3.569000
245  2017-09-17  3.542500
246  2017-09-24  3.507000
247  2017-10-01  3.530000

[248 rows x 2 columns]

First we take a look at the data description in orther to find the present quantity of null values

In [5]:
corn13_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   week    248 non-null    object 
 1   price   248 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.0+ KB


by the above we have a complete dataset. 

# Preprocess: Discretization

In particular, for the corn price predictor it is useful to model the corn prices as discrete observations rather than continous observations generated by a state dependent distribution (often a Gaussian).

## Generate Clusters

To simplify the problemwe will use clustering that works as a data quantization and this can simplifies the learning of an HMM. As each cluster label is going to be used as an observation. 

The emission probability matrix can be encoded as a discrete vector of probabilities.

In [6]:
def generate_cluster_assignments(ser, clusters):
    cluster = KMeans(n_clusters = clusters, random_state=24)
    df = pd.DataFrame(ser)
    cluster.fit(df)
    
    return cluster.predict(df)

# Estimating a Hidden Markov Model

In [7]:
corn13_17_seq = generate_cluster_assignments(corn13_17[['price']], 5)
corn13_17_seq

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0,
       0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 0, 0, 0, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3], dtype=int32)

## Number of Unique States

In [8]:
# Almost all functions require this constant as an argument
STATE_LIST = ['S1', 'S2']

# Initialze state transition probabilities (2 states)
STATE_TRANS_PROBS = [0.4, 0.6, 0.35, 0.55]

The following functions will be useful below

In [9]:
# given a list with unique items, this function will return a new list with all permutations
def make_state_permutations(list_of_unique_states):
    l1 = [''.join(tup) for tup in permutations(list_of_unique_states, 2)]
    l2 = [state+state for state in list_of_unique_states]
    return sorted(l1 + l2)

# helper function in EM function
def _grab_highest_prob_and_state(state_permutations_lst, prob_arr):
    return (prob_arr[np.argmax(prob_arr)], state_permutations_lst[np.argmax(prob_arr)])

and the following two transform a dictionary content into anew format

In [10]:
def dict_to_tuples(list_of_unique_states, d):
    
    # Defensive programming to ensure output will be correct
    list_of_unique_states = sorted(list_of_unique_states)
    assert make_state_permutations(list_of_unique_states) == list(d.keys()), \
            "Keys of dictionary must match output of `make_state_permutations(list_of_unique_states)`"
    
    lengths = [len(st) for st in list_of_unique_states]
    final_dict = {}
    for idx, st in enumerate(list_of_unique_states):
        tup = []
        for trans_p in d.keys():
            if trans_p[:lengths[idx]] == st:
                tup.append(d[trans_p])
            else:
                continue
        final_dict[st] = tuple(tup)
        
    return final_dict
    
def obs_to_tuples(list_of_unique_states, d, sequence):
    
    # Defensive programming to ensure output will be correct
    list_of_unique_states = sorted(list_of_unique_states)
    num_unique_obs = len(np.unique(sequence))
    
    lengths = [len(st) for st in list_of_unique_states]
    final_dict = {}
    for idx, st in enumerate(list_of_unique_states):
        tup = []
        for e_trans in d.keys():
            if e_trans[:lengths[idx]] == st:
                tup.append(d[e_trans])
            else:
                continue
        final_dict[st] = tuple(tup)
        
    return final_dict

# Define Transition Functions

Create our initial HMM parameters ($\pi$, $A$, $B$) in the form of a dictionary.

Dictionaries are more efficient than arrays, we will see it in the following. 

In [12]:
import timeit

all_setup = """

import numpy as np

# These hold the same information
arr = np.array([[0.4, 0.6], [0.35, 0.55]])
d = {'S1S1': 0.4, 'S1S2': 0.6, 'S2S1': 0.35, 'S2S2': 0.55}

"""
i = 10_000_000
index_an_array = 'arr[0,0]'
retrieve_value = "d['S1S1']"

print('Seconds to index an array {} times: {}'.format(
    i, timeit.timeit(setup=all_setup, stmt=index_an_array, number=i)))

print('\n','#' * 60, '\n')
print('Seconds to retrieve value {} times: {}'.format(i, timeit.timeit(setup=all_setup, stmt=retrieve_value, number=i)))

Seconds to index an array 10000000 times: 1.757321322002099

 ############################################################ 

Seconds to retrieve value 10000000 times: 0.2506234320026124


The following functions produce initial probabilities for ($\pi$, $A$, $B$)

In [13]:
def generate_state_trans_dict(list_of_unique_states, **kwargs):
    
    # number of states
    N = len(list_of_unique_states)
    
    # this runs if specific transitions are provided
    if kwargs:
        state_perms = [''.join(tup) for tup in permutations(list(kwargs.keys()), 2)]
        all_permutations = [state+state for state in list_of_unique_states] + state_perms
        pbs = chain.from_iterable(kwargs.values())
        state_trans_dict = {perm:p for perm, p in zip(sorted(all_permutations), pbs)}
        return state_trans_dict
    
    state_perms = [''.join(tup) for tup in permutations(list_of_unique_states, 2)]
    all_permutations = [state+state for state in list_of_unique_states] + state_perms
    state_trans_dict = {perm: (1/N) for perm in all_permutations}
    
    return state_trans_dict

In [21]:
def generate_emission_prob_dist(list_of_unique_states, sequence, **kwargs):
    
    # number of unique obs
    B = list(np.unique(sequence).astype(str))
    
    # this runs if specific transitions are provided
    if kwargs:
        
        for t in kwargs.values():
            
            assert len(t) == len(B)
            assert round(np.sum(t)) == 1.0
            
        for k in kwargs.keys():
            
            assert k in list_of_unique_states
            
        diff = list(set(list_of_unique_states).difference(kwargs.keys()))
        
        pbs = chain.from_iterable(kwargs.values())
        obs_perms = [state + '_' + str(obs) for state in kwargs.keys() for obs in B]
        
        obs_trans_dict = {perm:p for perm, p in zip(sorted(obs_perms), pbs)}
        
        if diff:
            obs_perms_diff = [state + '_' + obs for state in diff for obs in B]
            obs_trans_dict.update({perm: (1/len(B)) for perm in obs_perms_diff})
            
        return obs_trans_dict
    
    obs_perms = [state + '_' + obs for state in list_of_unique_states for obs in B]
    obs_trans_dict = {perm: (1/len(B)) for perm in obs_perms}
    
    return obs_trans_dict

In [22]:
def generate_init_prob_dist(list_of_unique_states, **kwargs):
    
    # number of states
    N = len(list_of_unique_states)
    
    # this runs if specific transitions are provided
    if kwargs:
        for t in kwargs.values():
            assert isinstance(t, float)
            assert t > 0
            
        assert np.sum(list(kwargs.values())) == 1.0
        assert len(kwargs) == len(list_of_unique_states)
        
        # build the prob dictionary
        init_prob_dict = {item[0]: item[1] for item in kwargs.items()}
        return init_prob_dict
    
    init_prob_dist = {state: (1/N) for state in list_of_unique_states}
    return init_prob_dist

# Create Transition Priors

In [23]:
# Make permutations of state transition (this len should match len(STATE_TRANS_PROBS))
state_transitions_list = make_state_permutations(STATE_LIST)

# Create transition matrix in form of dictionary
state_transition_probs = {
    trans: prob for trans, prob in zip(state_transitions_list, STATE_TRANS_PROBS)
}

state_transition_probs

{'S1S1': 0.4, 'S1S2': 0.6, 'S2S1': 0.35, 'S2S2': 0.55}

The dictionary shown above is the state transition "matrix" formated as a dictionary. 

In [24]:
# Transform dictionary to be in tuple format
A_prior = dict_to_tuples(STATE_LIST, state_transition_probs)
A_prior

{'S1': (0.4, 0.6), 'S2': (0.35, 0.55)}

#### NOTE ON FORMATTING

Some functions in this assignment require the transition probabilities be in a specific format. We will switch between two formats that hold the same information:
<br>

With 2 states:
<br>

**Format 1**: `{'S1S1': 0.4, 'S1S2': 0.6, 'S2S1': 0.35, 'S2S2': 0.55}`

The above dictionary is the state transition matrix in dictionary form, where every key is a transition from one state to another. For example, the key-value pair, `'S1S2': 0.6`, says the probability of the state moving from `S1` to `S2` from observation `i` to observation `j` is `0.6`.
<br>
<br>
**Format 2**: `{'S1': (0.4, 0.6), 'S2': (0.35, 0.55)}`    

The second format contains the same information as the first, but encodes the probabilities in tuples. With only two states and assuming `S1` is the first state, `'S1': (0.4, 0.6)` is interpreted as the probability of staying in `S1` is 0.4 and the probability of moving from `S1` to `S2` is 0.6. 

Below we will switch between both formats

# Create Emission Probability Priors

In [25]:
# Manually initialize emission probabilities - in format 1
B_format1 = {
    'S1_0': 0.1,
    'S1_1': 0.3,
    'S1_2': 0.4,
    'S1_3': 0.15,
    'S1_4': 0.05,
    'S2_0': 0.15,
    'S2_1': 0.2,
    'S2_2': 0.3,
    'S2_3': 0.05,
    'S2_4': 0.3
}

Convert emission matrix to format 2

In [26]:
B_format2 = obs_to_tuples(STATE_LIST, B_format1, corn13_17_seq)
B_format2

{'S1': (0.1, 0.3, 0.4, 0.15, 0.05), 'S2': (0.15, 0.2, 0.3, 0.05, 0.3)}

Convert B back to format 1

In [27]:
generate_emission_prob_dist(STATE_LIST, corn13_17_seq, **B_format2)

{'S1_0': 0.1,
 'S1_1': 0.3,
 'S1_2': 0.4,
 'S1_3': 0.15,
 'S1_4': 0.05,
 'S2_0': 0.15,
 'S2_1': 0.2,
 'S2_2': 0.3,
 'S2_3': 0.05,
 'S2_4': 0.3}

We will keep the emission probabilities in the `"key": tuple` format so that it can be used easily as a `**kwargs` argument later.

In [33]:
B_prior = obs_to_tuples(STATE_LIST, B_format1, corn13_17_seq)

In [36]:
# User specified initial probabilities
pi__init = {'S1': 0.4 , 'S2': 0.6}

# generate the dictionary holding initial state probabilities
pi = generate_init_prob_dist(STATE_LIST, **pi__init)
pi

{'S1': 0.4, 'S2': 0.6}

Finally, we need to create a data structure that will hold all of our probability calculations until we are finished computing

For this task, we will take advantage of a powerful data structure from the `collections` module `namedtuple`.

## NAMED TUPLES

Take a few minutes to review the [documentation](https://docs.python.org/3.6/library/collections.html#collections.namedtuple) on `namedtuples`. Then answer the following question.

Alternatively, for a short and helpful introduction, review [this tutorial](https://dbader.org/blog/writing-clean-python-with-namedtuples).

---

# Putting it all together

We now have all the pieces to use the EM algorithm. 

To do this requires the calculation of forward backward algorithm. We will use `namedtuple` from the `collections` module to do this.

The function below will generate this data structure.

In [29]:
def generate_obs_data_structure(sequence):
    
    #  sequence: 1D numpy array of observations
    ObservationData = namedtuple(
        'ObservationData',
        ['prob_lst', 'highest_prob', 'highest_state']
    )
    return {index+1: ObservationData for index in np.arange(len(sequence)-1)}

## ESTIMATE PROBABILITIES

This step involves using the Forward-Backward algorithm to calculate the probability of observing a sequence, given a set of HMM parameters. 

In [32]:
# Enforce an Argument Signature on following function to prevent errors with **kwargs
params = [Parameter('list_of_unique_states', Parameter.POSITIONAL_OR_KEYWORD),
         Parameter('sequence', Parameter.POSITIONAL_OR_KEYWORD),
         Parameter('A', Parameter.KEYWORD_ONLY, default=generate_state_trans_dict),
         Parameter('B', Parameter.KEYWORD_ONLY, default=generate_emission_prob_dist),
         Parameter('pi', Parameter.KEYWORD_ONLY, default=generate_init_prob_dist)]

sig = Signature(params)

def calculate_probabilities(list_of_unique_states, sequence, **kwargs):
    
    # enforce signature to ensure variable names
    bound_values = sig.bind(list_of_unique_states, sequence, **kwargs)
    bound_values.apply_defaults()

    
    # grab params that are left to default values
    param_defaults = [(name, val) for name, val in bound_values.arguments.items() if callable(val)]
    
    # grab non-default params
    set_params = [(name, val) for name, val in bound_values.arguments.items() if isinstance(val, dict)]
    
    # this will run if any default hmm parameters are used
    if param_defaults:
        for name, val in param_defaults:
            if name == 'B':
                B = val(list_of_unique_states, sequence)
            elif name == 'A':
                A = val(list_of_unique_states)
            elif name == 'pi':
                pi = val(list_of_unique_states)
            else:
                continue
    
    # this will run if kwargs are provided        
    if set_params:
        for name, val in set_params:
            if name == 'B':
                B = generate_emission_prob_dist(list_of_unique_states, sequence, **val)
            elif name == 'A':
                A = generate_state_trans_dict(list_of_unique_states, **val)
            elif name == 'pi':
                pi = generate_init_prob_dist(list_of_unique_states, **val)
            else:
                continue
                
    # instantiate the data structure
    obs_probs = generate_obs_data_structure(sequence)

    # all state transitions
    state_perms = make_state_permutations(list_of_unique_states)

    # for every transition from one observation to the next, calculate probability of going from Si to Sj
    # loop through observations
    for idx, obs in enumerate(sequence):

        if idx != 0:  # check if this is the first observation
            # instantiate the namedtuple for this observation
            obs_probs[idx] = obs_probs[idx]([], [], [])

            # loop through each possible state transition
            for st in state_perms:
                
                # calculate prob of current obs for this state
                prev_prob = pi[st[:2]] * B[st[:2]+'_'+str(sequence[idx-1])]

                # calculate prob of previous obs for this state
                curr_prob = A[st] * B[st[2:]+'_'+str(obs)]

                # combine these two probabilities
                combined_prob = round(curr_prob * prev_prob, 4)

                # append probability to the list in namedtuple
                obs_probs[idx].prob_lst.append(combined_prob)

            # check for highest prob of observing that sequence
            prob_and_state = _grab_highest_prob_and_state(state_perms, obs_probs[idx].prob_lst)
            obs_probs[idx].highest_prob.append(prob_and_state[0])
            obs_probs[idx].highest_state.append(prob_and_state[1])

        else: # this is the first observation, exit loop.
            continue
    return (obs_probs, A, B, pi)

In [37]:
ob_prob, A, B, pi = calculate_probabilities(STATE_LIST, corn13_17_seq, A=A_prior, B=B_prior, pi=pi)
ob_prob

{1: ObservationData(prob_lst=[0.0144, 0.0144, 0.0126, 0.0132], highest_prob=[0.0144], highest_state=['S1S1']),
 2: ObservationData(prob_lst=[0.0144, 0.0144, 0.0126, 0.0132], highest_prob=[0.0144], highest_state=['S1S1']),
 3: ObservationData(prob_lst=[0.0144, 0.0144, 0.0126, 0.0132], highest_prob=[0.0144], highest_state=['S1S1']),
 4: ObservationData(prob_lst=[0.0144, 0.0144, 0.0126, 0.0132], highest_prob=[0.0144], highest_state=['S1S1']),
 5: ObservationData(prob_lst=[0.0144, 0.0144, 0.0126, 0.0132], highest_prob=[0.0144], highest_state=['S1S1']),
 6: ObservationData(prob_lst=[0.0144, 0.0144, 0.0126, 0.0132], highest_prob=[0.0144], highest_state=['S1S1']),
 7: ObservationData(prob_lst=[0.0144, 0.0144, 0.0126, 0.0132], highest_prob=[0.0144], highest_state=['S1S1']),
 8: ObservationData(prob_lst=[0.0144, 0.0144, 0.0126, 0.0132], highest_prob=[0.0144], highest_state=['S1S1']),
 9: ObservationData(prob_lst=[0.0144, 0.0144, 0.0126, 0.0132], highest_prob=[0.0144], highest_state=['S1S1']),
 

In [38]:
# Using default initial parameters 
generate_init_prob_dist(STATE_LIST)

{'S1': 0.5, 'S2': 0.5}

In [39]:
A

{'S1S1': 0.4, 'S1S2': 0.6, 'S2S1': 0.35, 'S2S2': 0.55}

In [40]:
B

{'S1_0': 0.1,
 'S1_1': 0.3,
 'S1_2': 0.4,
 'S1_3': 0.15,
 'S1_4': 0.05,
 'S2_0': 0.15,
 'S2_1': 0.2,
 'S2_2': 0.3,
 'S2_3': 0.05,
 'S2_4': 0.3}

In [41]:
pi

{'S1': 0.4, 'S2': 0.6}

# Update Parameters

Update transition matrix

In [42]:
# This function sums all of the probabilities and 
# outputs a new (un-normalized) state transition matrix

def new_state_trans(STATE_LIST, probabilities):
    
    state_perms = make_state_permutations(STATE_LIST)
    sums_of_st_trans_prob = {p:0 for p in state_perms}
    highest_prob_sum = 0
    
    for obs in probabilities:
        highest_prob_sum += probabilities[obs].highest_prob[0]
        for i, p in enumerate(sums_of_st_trans_prob):
            sums_of_st_trans_prob[p] += probabilities[obs].prob_lst[i]
    
    for key in sums_of_st_trans_prob:
        sums_of_st_trans_prob[key] = sums_of_st_trans_prob[key] / highest_prob_sum
    
    # finally, normalize so the rows add up to 1
    for s in STATE_LIST:
        l = []
        for k in sums_of_st_trans_prob:
            if s == k[:2]:
                l.append(sums_of_st_trans_prob[k])
        for k in sums_of_st_trans_prob:
            if s == k[:2]:
                sums_of_st_trans_prob[k] = sums_of_st_trans_prob[k] / sum(l)
    
    return sums_of_st_trans_prob

Update and normalize posterior state transition

In [43]:
A_posterior = new_state_trans(STATE_LIST, ob_prob)
A_posterior

{'S1S1': 0.4602152515867919,
 'S1S2': 0.5397847484132082,
 'S2S1': 0.3367172332093854,
 'S2S2': 0.6632827667906147}

Convert state transition to "format 2" so it can be used as input in the next iteration of "E" step

In [44]:
A_posterior = dict_to_tuples(STATE_LIST, A_posterior)

## Update The Emission Probability

Here, we define some functions designed to do specific tasks.

In [46]:
##### tally up all observed sequences
def observed_pairs(sequence):
    observed_pairs = []
    
    for idx in range(len(sequence)-1):
        observed_pairs.append((sequence[idx], sequence[idx+1]))
        
    return observed_pairs

def make_emission_permutations(sequence):
    
    unique_e = np.unique(sequence)
    
    return list(product(unique_e, repeat = 2))

def find_highest_with_state_obs(prob_pairs, state, obs):
    
    for pp in prob_pairs:
        if pp[0].count((state,obs))>0:
            
            return pp[1]
        
def normalize_emissions(b_tuple_format):
    
    new_b_dict = {}
    
    for key, val in b_tuple_format.items():
        denominator = sum(val)
        new_lst = [v/denominator for v in val]
        new_b_dict[key] = tuple(new_lst)
        
    return new_b_dict

make_emission_permutations([1,1,0, 2])
make_emission_permutations([0,1,0,3,0])

[(0, 0), (0, 1), (0, 3), (1, 0), (1, 1), (1, 3), (3, 0), (3, 1), (3, 3)]

We update the emissions probabilities

In [48]:
def emission_matrix_update(sequence, state_list, A, B, pi):
    state_pairs = list(product(state_list, repeat = 2))
    obs_pairs = observed_pairs(sequence)
    
    new_B = {}
    for obs in np.unique(sequence): # For every unique emission
        
        # Find all the sequence-pairs that include that emission
        inc_seq = [seq for seq in obs_pairs if seq.count(obs)>0]

        # Collector for highest-probabilities
        highest_pairs = []
        
        # For each sequence-pair that include that emission
        for seq in inc_seq:

            prob_pairs = []
            
            # Go through each potential pair of states
            for state_pair in state_pairs:
                
                state1, state2 = state_pair
                obs1, obs2 = seq
                
                # Match each state with it's emission
                assoc_tuples = [(state1, obs1),
                                (state2, obs2)]
                
                # Calculate the probability of the sequence from state
                prob = pi[state1] * B[state1+"_"+str(obs1)]
                prob *= A[state1+state2]*B[state2+"_"+str(obs2)]
                prob = round(prob,5)
                # Append the state emission tuples and probability
                prob_pairs.append([assoc_tuples, prob])
    
            # Sort probabilities by maximum probability
            prob_pairs = sorted(prob_pairs, key = lambda x: x[1], reverse = True)
            
            # Save the highest probability
            to_add = {'highest':prob_pairs[0][1]}
            # Find the highest probability where each state is associated
            # With the current emission
            for state in STATE_LIST:
                
                highest_of_state = 0
                
                # Go through sorted list, find first (state,observation) tuple
                # save associated probability

                for pp in prob_pairs:
                    if pp[0].count((state,obs))>0:
                        highest_of_state = pp[1]
                        break
                        
                to_add[state] = highest_of_state
            
            # Save completed dictionary
            highest_pairs.append(to_add)
        
        # Total highest_probability
        highest_probability =sum([d['highest'] for d in highest_pairs])
        
        # Total highest probabilities for each state; divide by highest prob
        # Add to new emission matrix
        for state in STATE_LIST:
            new_B[state+"_"+str(obs)]= sum([d[state] for d in highest_pairs])/highest_probability
            
        
    return new_B

In [49]:
nb = emission_matrix_update(corn13_17_seq,STATE_LIST, A,B,pi)
nb

{'S1_0': 0.47605061556159334,
 'S2_0': 1.0,
 'S1_1': 0.9985815602836879,
 'S2_1': 0.9914893617021275,
 'S1_2': 0.9674200176108009,
 'S2_2': 1.0,
 'S1_3': 1.0,
 'S2_3': 0.50093808630394,
 'S1_4': 0.12074074074074062,
 'S2_4': 1.0}

Normalize the updated emission probabilities

In [50]:
B_ = obs_to_tuples(STATE_LIST, nb, corn13_17_seq)
B_posterior = normalize_emissions(B_)

In [51]:
# normalized state transition posterior:
A_posterior

{'S1': (0.4602152515867919, 0.5397847484132082),
 'S2': (0.3367172332093854, 0.6632827667906147)}

In [52]:
# normalized emission posterior probabilities
B_posterior

{'S1': (0.13361725599944574,
  0.2802805492003152,
  0.27153416869254315,
  0.28067867498042925,
  0.03388935112726661),
 'S2': (0.2225968057522761,
  0.22070236485225672,
  0.2225968057522761,
  0.11150721789091506,
  0.2225968057522761)}

## Repeat Until Parameters Converge

In [53]:
ob_prob2, A2, B2, pi2 = calculate_probabilities(STATE_LIST, corn13_17_seq, A=A_posterior, B=B_posterior, pi=pi)
ob_prob2

{1: ObservationData(prob_lst=[0.0145, 0.0134, 0.0125, 0.0194], highest_prob=[0.0194], highest_state=['S2S2']),
 2: ObservationData(prob_lst=[0.0145, 0.0134, 0.0125, 0.0194], highest_prob=[0.0194], highest_state=['S2S2']),
 3: ObservationData(prob_lst=[0.0145, 0.0134, 0.0125, 0.0194], highest_prob=[0.0194], highest_state=['S2S2']),
 4: ObservationData(prob_lst=[0.0145, 0.0134, 0.0125, 0.0194], highest_prob=[0.0194], highest_state=['S2S2']),
 5: ObservationData(prob_lst=[0.0145, 0.0134, 0.0125, 0.0194], highest_prob=[0.0194], highest_state=['S2S2']),
 6: ObservationData(prob_lst=[0.0145, 0.0134, 0.0125, 0.0194], highest_prob=[0.0194], highest_state=['S2S2']),
 7: ObservationData(prob_lst=[0.0145, 0.0134, 0.0125, 0.0194], highest_prob=[0.0194], highest_state=['S2S2']),
 8: ObservationData(prob_lst=[0.0145, 0.0134, 0.0125, 0.0194], highest_prob=[0.0194], highest_state=['S2S2']),
 9: ObservationData(prob_lst=[0.0145, 0.0134, 0.0125, 0.0194], highest_prob=[0.0194], highest_state=['S2S2']),
 

In [54]:
# update and normalize state transition matrix again
A_post2 = new_state_trans(STATE_LIST, ob_prob2)  
# convert to `key: tuple` format
A_post2 = dict_to_tuples(STATE_LIST, A2)  
A_post2

{'S1': (0.4602152515867919, 0.5397847484132082),
 'S2': (0.3367172332093854, 0.6632827667906147)}

In [55]:
# update emissions matrix again
nb2 = emission_matrix_update(corn13_17_seq, STATE_LIST, A2, B2, pi)  # update emissions matrix again
B_post2 = obs_to_tuples(STATE_LIST, nb2, corn13_17_seq)  # convert emission posterior to `key:tuples` format
B_post2 = normalize_emissions(B_post2)  # normalize emissions probabilities
B_post2

{'S1': (0.11428246205333821,
  0.26164105059540205,
  0.24272670502233154,
  0.3521679593360904,
  0.02918182299283783),
 'S2': (0.22369084423305585,
  0.22369084423305585,
  0.22369084423305585,
  0.10523662306777645,
  0.22369084423305585)}